In [9]:
import pandas as pd
from catboost import CatBoostClassifier
import json
from sklearn.model_selection import train_test_split
from collections import Counter
pd.set_option('max_columns', None)

In [2]:
train = pd.read_csv('../data/train.csv')
label = pd.read_csv('../data/train_label.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')
train_data = pd.merge(train,label,on='ID')



/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_data.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数,Label
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,...,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0,0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10,0.0,42:05.0,9.0,0.0,266000,NaN,19.160223,00:00.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,00:00.0,5439.0,...,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,319.205293,0.0,0


In [4]:
def deal_time(x):
    if str(x) == 'nan':
        return x
    
    x = str(x)
    parts = x.split(':')
    return int(parts[0]) + float(parts[1])/100

In [5]:
def cal_diff(x, y):
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    else:
        return y-x

In [6]:
def deal_manage_range(x):
    g = json.loads(x)
    return len(g)

In [7]:
all_ranges = []
jyfw = list(train_data['经营范围'])
for item in jyfw:
    all_ranges += json.loads(item)
range_dict = dict(Counter(all_ranges))
# bins = [4,8,16,64,128,256,600,3000]
def deal_manage_range_bin(x, lower, upper=100000):
    count = 0
    items = json.loads(x)
    for item in items:
        counts = range_dict.get(item)
        if counts<=upper and counts>lower:
            count += 1
    return count

In [8]:
train_data['经营期限至'] = train_data['经营期限至'].apply(deal_time)
train_data['经营期限自'] = train_data['经营期限自'].apply(deal_time)
train_data['成立日期'] = train_data['成立日期'].apply(deal_time)
train_data['核准日期'] = train_data['核准日期'].apply(deal_time)
train_data['注销时间'] = train_data['注销时间'].apply(deal_time)

train_data['经营范围1'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
train_data['经营范围2'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
train_data['经营范围3'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(8,16,))
train_data['经营范围4'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(16,64,))
train_data['经营范围5'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(64,128,))
train_data['经营范围6'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(128,256,))
train_data['经营范围7'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(256,600,))
train_data['经营范围8'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(600,3000,))
train_data['经营范围9'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(3000,))

train_data['经营范围'] = train_data['经营范围'].apply(deal_manage_range)

for item in train_data.columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            train_data[parts[0] + '_diff'] = train_data.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

In [10]:
train_data.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,注销时间,经营期限自,成立日期,行业门类,企业类别,管辖机关,经营范围,增值税,企业所得税,印花税,教育费,城建税,递延收益_年初数,长期负债合计_年初数,长期借款_年初数,长期应付款_年初数,短期借款_年初数,递延所得税负债_年初数,非流动负债合计_年初数,负债合计_年初数,负债和所有者权益总计_年初数,交易性金融负债_年初数,流动负债合计_年初数,其他非流动负债_年初数,其他负债（或长期负债）_年初数,其他流动负债_年初数,其他应付款_年初数,其他应交款_年初数,实收资本（股本）_年初数,实收资本（或股本）净额_年初数,所有者权益合计_年初数,未分配利润_年初数,小企业_应付利润_年初数,应付福利费_年初数,应付股利_年初数,应付利息_年初数,应付票据_年初数,应付职工薪酬_年初数,应付债券_年初数,应付账款_年初数,预计负债_年初数,应交税费_年初数,一年内到期的非流动负债_年初数,预提费用_年初数,预收款项_年初数,盈余公积_年初数,资本公积_年初数,专项应付款_年初数,递延收益_年末数,长期负债合计_年末数,长期借款_年末数,长期应付款_年末数,短期借款_年末数,递延所得税负债_年末数,非流动负债合计_年末数,负债合计_年末数,负债和所有者权益总计_年末数,交易性金融负债_年末数,流动负债合计_年末数,其他非流动负债_年末数,其他负债（或长期负债）_年末数,其他流动负债_年末数,其他应付款_年末数,其他应交款_年末数,实收资本（股本）_年末数,实收资本（或股本）净额_年末数,所有者权益合计_年末数,未分配利润_年末数,小企业_应付利润_年末数,应付福利费_年末数,应付股利_年末数,应付利息_年末数,应付票据_年末数,应付职工薪酬_年末数,应付债券_年末数,应付账款_年末数,预计负债_年末数,应交税费_年末数,一年内到期的非流动负债_年末数,预提费用_年末数,预收款项_年末数,盈余公积_年末数,资本公积_年末数,专项应付款_年末数,货币资金_年初数,存货_年初数,存货中的原材料_年初数,存货中的周转材料_年初数,长期待摊费用_年初数,长期股权投资_年初数,长期投资合计_年初数,长期应收款_年初数,持有至到期投资_年初数,短期投资_年初数,待摊费用_年初数,递延所得税资产_年初数,非流动资产合计_年初数,工程物资_年初数,固定资产合计_年初数,固定资产净额_年初数,固定资产净值_年初数,固定资产清理_年初数,固定资产原价_年初数,交易性金融资产_年初数,库存商品(产成品)_年初数,开发支出_年初数,可供出售金融资产_年初数,流动资产合计_年初数,其他非流动资产_年初数,其他流动资产_年初数,其他应收款_年初数,生产性生物资产_年初数,商誉_年初数,投资性房地产_年初数,无形资产_年初数,无形资产及其他资产合计_年初数,应收账款_年初数,一年内到期的非流动资产_年初数,油气资产_年初数,应收补贴款_年初数,应收股利_年初数,应收利息_年初数,应收票据_年初数,预付款项_年初数,资产总计_年初数,在建工程_年初数,货币资金_年末数,存货_年末数,存货中的原材料_年末数,存货中的周转材料_年末数,长期待摊费用_年末数,长期股权投资_年末数,长期投资合计_年末数,长期应收款_年末数,持有至到期投资_年末数,短期投资_年末数,待摊费用_年末数,递延所得税资产_年末数,非流动资产合计_年末数,工程物资_年末数,固定资产合计_年末数,固定资产净额_年末数,固定资产净值_年末数,固定资产清理_年末数,固定资产原价_年末数,交易性金融资产_年末数,库存商品(产成品)_年末数,开发支出_年末数,可供出售金融资产_年末数,流动资产合计_年末数,其他非流动资产_年末数,其他流动资产_年末数,其他应收款_年末数,生产性生物资产_年末数,商誉_年末数,投资性房地产_年末数,无形资产_年末数,无形资产及其他资产合计_年末数,应收账款_年末数,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数,Label,经营范围1,经营范围2,经营范围3,经营范围4,经营范围5,经营范围6,经营范围7,经营范围8,经营范围9,递延收益_diff,长期负债合计_diff,长期借款_diff,长期应付款_diff,短期借款_diff,递延所得税负债_diff,非流动负债合计_diff,负债合计_diff,负债和所有者权益总计_diff,交易性金融负债_diff,流动负债合计_diff,其他非流动负债_diff,其他负债（或长期负债）_diff,其他流动负债_diff,其他应付款_diff,其他应交款_diff,实收资本（股本）_diff,实收资本（或股本）净额_diff,所有者权益合计_diff,未分配利润_diff,应付福利费_diff,应付股利_diff,应付利息_diff,应付票据_diff,应付职工薪酬_diff,应付债券_diff,应付账款_diff,预计负债_diff,应交税费_diff,一年内到期的非流动负债_diff,预提费用_diff,预收款项_diff,盈余公积_diff,资本公积_diff,专项应付款_diff,货币资金_diff,存货_diff,存货中的原材料_diff,存货中的周转材料_diff,长期待摊费用_diff,长期股权投资_diff,长期投资合计_diff,长期应收款_diff,持有至到期投资_diff,短期投资_diff,待摊费用_diff,递延所得税资产_diff,非流动资产合计_diff,工程物资_diff,固定资产合计_diff,固定资产净额_diff,固定资产净值_diff,固定资产清理_diff,固定资产原价_diff,交易性金融资产_diff,库存商品(产成品)_diff,开发支出_diff,可供出售金融资产_diff,流动资产合计_diff,其他非流动资产_diff,其他流动资产_diff,其他应收款_diff,生产性生物资产_diff,商誉_diff,投资性房地产_diff,无形资产_diff,无形资产及其他资产合计_diff,应收账款_diff,一年内到期的非流动资产_diff,油气资产_diff,应收补贴款_diff,应收股利_diff,应收利息_diff,应收票据_diff,预付款项_diff,资产总计_diff,在建工程_diff
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,0.00,5154.0,NaN,0.00,0.00,1.0,0.0,1.0,37,106.039650,0.331552,1.247218,3.181189,7.422776,0.0,0.0,25.110484,0.0,0.0,0.0,25.110484,797.386637,844.162546,0.0,772.276153,0.0,0.0,0.0,106.078436,0.0,50.220972,0.0,46.775910,-3.445063,0.0,0.0,0.0,0.0,125.552431,0.000000,0.0,520.839979,0.0,19.805307,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,585.013852,632.032182,0.0,585.013852,0.0,0.0,0.0,83.123866,0.0,50.220972,0.0,47.018330,-3.202643,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,489.934244,0.0,11.955743,0.0,0.0,0.000000,0.0,0.0,0.0,24.579246,462.290647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135.081011,0.0,135.081011,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,709.081535,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,215.371727,0.0,0.0,0.0,0.0,0.0,2.647788,4.192126,844.162546,0.0,31.416284,229.147518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.128884,0.0,0.0,99.813781,0.0,99.813781,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,532.218401,0.0,0.0,20.088389,0.0,0.0,0.0,0.0,0.0,142.310005,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0,0,10,4,7,11,1,0,1,0,3,0.0,0.0,-25.110484,0.0,0.0,0.0,-25.110484,-212.372784,-212.130364,0.0,-187.262300,0.0,0.0,0.0,-22.954570,0.0,0.0,0.0,0.242420,0.242420,0.0,0.0,0.0,-125.552431,0.000000,0.0,-30.9057

In [10]:
drop_feat = ['ID','邮政编码','Label']
feat = [item for item in train_data.columns if item not in drop_feat]

X = train_data[feat]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [16]:
from sklearn.metrics import roc_auc_score
prob_oof = cat_clf.predict_proba(x_test)
prob_oof = [item[1] for item in prob_oof]
score = roc_auc_score(y_test, prob_oof)
score

0.9232948913227436